# Create land binary masks

In [ ]:
import ants
import iris

import matplotlib.pyplot as plt
# import numpy as np
from aeolus.io import create_dummy_cube
from cf_units import Unit
from iris.analysis.cartography import get_xy_grids, wrap_lons

In [ ]:
import paths
from common import B2_CONT, B8_CONT, E4_CONT

In [ ]:
tcoord = iris.coords.AuxCoord(
    points=0,
    units=Unit("hours since 1970-01-01 00:00:00", calendar="gregorian"),
    standard_name="time",
)

In [ ]:
resolutions = [
    96,
]  # 320, 512, 768, 1024]

In [ ]:
for n_res in resolutions:

    cube = create_dummy_cube(n_res=n_res)
    cube.add_aux_coord(tcoord)

    lons, lats = get_xy_grids(cube)
    lons_pm180 = wrap_lons(lons, -180, 360)
    # B2
    label = "b2"
    lon0 = -36.25
    lon1 = 36.25
    lat0 = -19
    lat1 = 19
    # B8
    # label = "b8"
    # lon0 = -93.75
    # lon1 = 93.75
    # lat0 = -49
    # lat1 = 49
    # E4
    # label = "e4"
    # lon0 = 1.25
    # lon1 = 113.75
    # lat0 = -29
    # lat1 = 29

    mask_true = (lons_pm180 >= lon0) & (lons_pm180 <= lon1) & (lats >= lat0) & (lats <= lat1)

    cube.data[mask_true] = 1.0

    cube.attributes["STASH"] = iris.fileformats.pp.STASH.from_msi("m01s00i030")
    cube.attributes["grid_staggering"] = 6
    cube.rename("land_binary_mask")

    outdir = paths.data / "land_sea_mask" / f"n{n_res}"
    outdir.mkdir(exist_ok=True, parents=True)
    fname = outdir / f"land_mask_lewis18_{label.lower()}_n{n_res}_{mask_true.sum()}p.anc"
    ants.save(cube, str(fname), saver="ancil")
    print(f"Saved to {fname}")